In [1]:
import pickle

In [2]:
with open('/project_final/orbitals.pkl', 'rb') as f:
    orbitals = pickle.load(f)

with open('/project_final/maneuvers.pkl', 'rb') as f:
    maneuvers = pickle.load(f)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

In [4]:
man = maneuvers['srl']

In [5]:
df = orbitals['SARAL.csv'].copy()

In [6]:
df.index = pd.to_datetime(df.index)

# keep only year-month-day
df.index = df.index.normalize()

In [7]:
import tensorflow as tf
import random
from itertools import product
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [8]:
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(42)

In [9]:
def create_sequences(data, window_size):
    sequences, targets = [], []
    for i in range(len(data) - window_size):
        sequences.append(data[i:i+window_size])
        targets.append(data[i+window_size][0])
    return np.array(sequences), np.array(targets)

In [10]:
def create_lstm_model(window_size, n_features, learning_rate, hidden_units, dropout_rate, fc_size):
    model = Sequential()
    model.add(Input(shape=(window_size, n_features)))
    model.add(LSTM(hidden_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(fc_size, activation='tanh'))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mae')
    return model

In [11]:
def plot_residuals_with_maneuvers(residual_all, residual_index, man_dates):
    man_dates_sorted = pd.to_datetime(man_dates).sort_values()
    lstm_ae_df = pd.DataFrame({'date': residual_index, 'residual': residual_all})

    plt.figure(figsize=(8, 4))
    plt.plot(lstm_ae_df["date"], lstm_ae_df["residual"], label="LSTM-AE Absolute Residual", color='blue')

    for d in man_dates_sorted:
        plt.axvline(d, color='gray', linestyle='--', alpha=0.3)

    plt.xlabel("Date")
    plt.ylabel("Abs Residual")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [12]:
def convert_timestamp_series_to_epoch(series):
    return (
        (series - pd.Timestamp(year=1970, month=1, day=1)) // pd.Timedelta(seconds=1)
    ).values

In [13]:
def compute_simple_matching_precision_recall_for_one_threshold(
    matching_max_days,
    threshold,
    series_ground_truth_manoeuvre_timestamps,
    series_predictions,
):
    matching_max_distance_seconds = pd.Timedelta(days=matching_max_days).total_seconds()
    dict_predictions_to_ground_truth = {}
    dict_ground_truth_to_predictions = {}

    manoeuvre_timestamps_seconds = convert_timestamp_series_to_epoch(series_ground_truth_manoeuvre_timestamps)
    pred_time_stamps_seconds = convert_timestamp_series_to_epoch(series_predictions.index)
    predictions = series_predictions.to_numpy()

    for i in range(predictions.shape[0]):
        if predictions[i] >= threshold:
            left_index = np.searchsorted(
                manoeuvre_timestamps_seconds, pred_time_stamps_seconds[i]
            )

            if left_index != 0:
                left_index -= 1

            index_of_closest = left_index

            if (left_index < len(series_ground_truth_manoeuvre_timestamps) - 1) and (
                abs(manoeuvre_timestamps_seconds[left_index] - pred_time_stamps_seconds[i])
                > abs(manoeuvre_timestamps_seconds[left_index + 1] - pred_time_stamps_seconds[i])
            ):
                index_of_closest = left_index + 1

            diff = abs(manoeuvre_timestamps_seconds[index_of_closest] - pred_time_stamps_seconds[i])

            if diff < matching_max_distance_seconds:
                dict_predictions_to_ground_truth[i] = (index_of_closest, diff)
                if index_of_closest in dict_ground_truth_to_predictions:
                    dict_ground_truth_to_predictions[index_of_closest].append(i)
                else:
                    dict_ground_truth_to_predictions[index_of_closest] = [i]

    positive_prediction_indices = np.argwhere(predictions >= threshold)[:, 0]
    list_false_positives = [
        pred_ind for pred_ind in positive_prediction_indices if pred_ind not in dict_predictions_to_ground_truth
    ]
    list_false_negatives = [
        true_ind for true_ind in np.arange(0, len(series_ground_truth_manoeuvre_timestamps))
        if true_ind not in dict_ground_truth_to_predictions
    ]

    precision = len(dict_ground_truth_to_predictions) / (len(dict_ground_truth_to_predictions) + len(list_false_positives)) if (len(dict_ground_truth_to_predictions) + len(list_false_positives)) > 0 else 0.0
    recall = len(dict_ground_truth_to_predictions) / (len(dict_ground_truth_to_predictions) + len(list_false_negatives)) if (len(dict_ground_truth_to_predictions) + len(list_false_negatives)) > 0 else 0.0

    return (precision, recall)

In [14]:
def f_beta_score(precision, recall, beta=2):
    beta_sq = beta ** 2
    denom = (beta_sq * precision) + recall
    if denom == 0:
        return 0
    return (1 + beta_sq) * precision * recall / denom

In [15]:
man_dates = man['median_day_time'].sort_values()

In [16]:
FIXED_WINDOW = 3 
TRAIN_FRAC, VAL_FRAC, TEST_FRAC = 0.6, 0.2, 0.2 

def time_split_index(idx, train_frac=0.7, val_frac=0.15):
    n = len(idx)
    i_train = int(n * train_frac)
    i_val = int(n * (train_frac + val_frac))
    return idx[:i_train], idx[i_train:i_val], idx[i_val:]

train_idx, val_idx, test_idx = time_split_index(df.index, TRAIN_FRAC, VAL_FRAC)

df_train = df.loc[train_idx]
df_val   = df.loc[val_idx]
df_test  = df.loc[test_idx]

man_dates_train = man_dates[(man_dates >= train_idx.min()) & (man_dates <= train_idx.max())]
man_dates_val   = man_dates[(man_dates >= val_idx.min()) & (man_dates <= val_idx.max())]
man_dates_test  = man_dates[(man_dates >= test_idx.min()) & (man_dates <= test_idx.max())]

In [17]:

from sklearn.preprocessing import MinMaxScaler

def make_sequences_1d(series_1d, window_size):
    X, y = create_sequences(series_1d.reshape(-1, 1), window_size)
    # X shape: (N-window, window_size, 1); y: (N-window,)
    return X, y

def train_on_train_return_residuals(df_train, df_val, df_test, params, epochs=20):

    scaler = MinMaxScaler()
    train_y = df_train['Brouwer mean motion'].values.reshape(-1, 1)
    scaler.fit(train_y)

    y_tr = scaler.transform(train_y)
    y_va = scaler.transform(df_val['Brouwer mean motion'].values.reshape(-1,1))
    y_te = scaler.transform(df_test['Brouwer mean motion'].values.reshape(-1,1))

    X_tr, y_tr_next = make_sequences_1d(y_tr, params['window_size'])
    X_va, y_va_next = make_sequences_1d(y_va, params['window_size'])
    X_te, y_te_next = make_sequences_1d(y_te, params['window_size'])

    set_seed(42)
    model = create_lstm_model(
        window_size=params['window_size'],
        n_features=1,
        learning_rate=params['learning_rate'],
        hidden_units=params['hidden_units'],
        dropout_rate=params['dropout_rate'],
        fc_size=params['fc_size']
    )
    model.fit(X_tr, y_tr_next, epochs=epochs, batch_size=32, shuffle=False, verbose=0)

    res_tr = np.abs(y_tr_next - model.predict(X_tr, verbose=0).flatten())
    res_va = np.abs(y_va_next - model.predict(X_va, verbose=0).flatten())
    res_te = np.abs(y_te_next - model.predict(X_te, verbose=0).flatten())

    idx_tr = df_train.index[params['window_size']:]
    idx_va = df_val.index[params['window_size']:]
    idx_te = df_test.index[params['window_size']:]

    ser_tr = pd.Series(res_tr, index=idx_tr, name='residual')
    ser_va = pd.Series(res_va, index=idx_va, name='residual')
    ser_te = pd.Series(res_te, index=idx_te, name='residual')

    return model, scaler, ser_tr, ser_va, ser_te

param_grid = {
    'window_size': [FIXED_WINDOW],      
    'learning_rate': [0.01, 0.001],
    'hidden_units': [16, 32, 64, 128],
    'dropout_rate': [0.0, 0.3, 0.5],
    'fc_size': [8, 16, 32, 64],
}

def evaluate_on_split(residual_series, man_dates_split, match_days=3, thresholds=None):
    if thresholds is None:
        thresholds = np.linspace(residual_series.min(), residual_series.max(), 500)
    precisions, recalls = [], []
    for thr in thresholds:
        p, r = compute_simple_matching_precision_recall_for_one_threshold(
            matching_max_days=match_days,
            threshold=thr,
            series_ground_truth_manoeuvre_timestamps=man_dates_split,
            series_predictions=residual_series
        )
        precisions.append(p); recalls.append(r)
    precisions = np.array(precisions); recalls = np.array(recalls)
    f1 = 2*precisions*recalls/(precisions+recalls+1e-12)
    return thresholds, precisions, recalls, f1

search_records = []
for lr in param_grid['learning_rate']:
    for hu in param_grid['hidden_units']:
        for dr in param_grid['dropout_rate']:
            for fc in param_grid['fc_size']:
                params = dict(window_size=FIXED_WINDOW, learning_rate=lr,
                              hidden_units=hu, dropout_rate=dr, fc_size=fc)
                print("Searching params:", params)
                model, scaler, ser_tr, ser_va, ser_te = train_on_train_return_residuals(
                    df_train, df_val, df_test, params, epochs=20
                )
                thr, P, R, F1 = evaluate_on_split(ser_va, man_dates_val, match_days=3)
                best_idx = np.argmax(F1)
                search_records.append({
                    'params': params,
                    'best_thr_on_val': float(thr[best_idx]),
                    'best_f1_on_val': float(F1[best_idx]),
                    'P_on_val': float(P[best_idx]),
                    'R_on_val': float(R[best_idx]),

                    'ser_tr': ser_tr, 'ser_va': ser_va, 'ser_te': ser_te,
                })


search_df = pd.DataFrame([{k:v for k,v in rec.items() if k not in ['ser_tr','ser_va','ser_te']} for rec in search_records])
best_rec = max(search_records, key=lambda r: r['best_f1_on_val'])
best_params = best_rec['params']
best_thr = best_rec['best_thr_on_val']
print("Best params (by val F1):", best_params, "Best thr:", best_thr)


df_trval = pd.concat([df_train, df_val])

def train_on_trval_and_get_residuals(df_trval, df_test, params, epochs=20):
    scaler = MinMaxScaler()
    y_trval = df_trval['Brouwer mean motion'].values.reshape(-1,1)
    scaler.fit(y_trval)

    y_trval_s = scaler.transform(y_trval)
    y_test_s  = scaler.transform(df_test['Brouwer mean motion'].values.reshape(-1,1))

    X_trv, y_trv_next = make_sequences_1d(y_trval_s, params['window_size'])
    X_tst, y_tst_next = make_sequences_1d(y_test_s, params['window_size'])

    set_seed(42)
    model = create_lstm_model(
        window_size=params['window_size'],
        n_features=1,
        learning_rate=params['learning_rate'],
        hidden_units=params['hidden_units'],
        dropout_rate=params['dropout_rate'],
        fc_size=params['fc_size']
    )
    model.fit(X_trv, y_trv_next, epochs=epochs, batch_size=32, shuffle=False, verbose=0)

    res_trv = np.abs(y_trv_next - model.predict(X_trv, verbose=0).flatten())
    res_tst = np.abs(y_tst_next - model.predict(X_tst, verbose=0).flatten())

    idx_trv = df_trval.index[params['window_size']:]
    idx_tst = df_test.index[params['window_size']:]

    ser_trv = pd.Series(res_trv, index=idx_trv, name='residual')
    ser_tst = pd.Series(res_tst, index=idx_tst, name='residual')
    return model, scaler, ser_trv, ser_tst

final_model, final_scaler, ser_trval, ser_test = train_on_trval_and_get_residuals(
    df_trval, df_test, best_params, epochs=20
)

def precision_recall_at_threshold(residual_series, man_dates_split, thr, match_days=3):
    p, r = compute_simple_matching_precision_recall_for_one_threshold(
        matching_max_days=match_days,
        threshold=thr,
        series_ground_truth_manoeuvre_timestamps=man_dates_split,
        series_predictions=residual_series
    )
    f1 = 2*p*r/(p+r+1e-12)
    return p, r, f1

ser_train_best = best_rec['ser_tr']
ser_val_best   = best_rec['ser_va']
ser_test_final = ser_test  

p_tr, r_tr, f1_tr = precision_recall_at_threshold(ser_train_best, man_dates_train, best_thr)
p_va, r_va, f1_va = precision_recall_at_threshold(ser_val_best,   man_dates_val,   best_thr)
p_te, r_te, f1_te = precision_recall_at_threshold(ser_test_final, man_dates_test,  best_thr)

ser_all = pd.concat([ser_train_best, ser_val_best, ser_test_final]).sort_index()
man_all = pd.concat([man_dates_train, man_dates_val, man_dates_test]).sort_values()

p_all, r_all, f1_all = precision_recall_at_threshold(ser_all, man_all, best_thr)

metrics_table = pd.DataFrame([
    {'split':'train', 'precision':p_tr, 'recall':r_tr, 'f1':f1_tr, 'threshold':best_thr},
    {'split':'val',   'precision':p_va, 'recall':r_va, 'f1':f1_va, 'threshold':best_thr},
    {'split':'test',  'precision':p_te, 'recall':r_te, 'f1':f1_te, 'threshold':best_thr},
    {'split':'ALL(micro)', 'precision':p_all, 'recall':r_all, 'f1':f1_all, 'threshold':best_thr},
])
print(metrics_table)

Searching params: {'window_size': 3, 'learning_rate': 0.01, 'hidden_units': 16, 'dropout_rate': 0.0, 'fc_size': 8}
Searching params: {'window_size': 3, 'learning_rate': 0.01, 'hidden_units': 16, 'dropout_rate': 0.0, 'fc_size': 16}
Searching params: {'window_size': 3, 'learning_rate': 0.01, 'hidden_units': 16, 'dropout_rate': 0.0, 'fc_size': 32}
Searching params: {'window_size': 3, 'learning_rate': 0.01, 'hidden_units': 16, 'dropout_rate': 0.0, 'fc_size': 64}
Searching params: {'window_size': 3, 'learning_rate': 0.01, 'hidden_units': 16, 'dropout_rate': 0.3, 'fc_size': 8}
Searching params: {'window_size': 3, 'learning_rate': 0.01, 'hidden_units': 16, 'dropout_rate': 0.3, 'fc_size': 16}
Searching params: {'window_size': 3, 'learning_rate': 0.01, 'hidden_units': 16, 'dropout_rate': 0.3, 'fc_size': 32}
Searching params: {'window_size': 3, 'learning_rate': 0.01, 'hidden_units': 16, 'dropout_rate': 0.3, 'fc_size': 64}
Searching params: {'window_size': 3, 'learning_rate': 0.01, 'hidden_units'